In [ ]:
# %% Deep learning - Section 11.108
#    Code challenge 14: binarised MNIST images

#    1) Start drom code from video 11.107
#    2) Use the same model architecture and metaparameters
#    3) Binarize the images so that each pixel is either 1 or 0
#    4) Test the model with these modified data
#    5) Optional: use different binarisation strategies (e.g. median split or
#       greater than 0)

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [ ]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import time

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [ ]:
# %% Data

# Load data
data = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'),delimiter=',')

# Split labels from data
labels = data[:,0]
data   = data[:,1:]

# Normalise data (original range is (0,255))
data_norm = data / np.max(data)


In [ ]:
# %% Binarise the normalise data

split_val = 0.5
data_bin  = np.where(data_norm > split_val,1,0)


In [ ]:
# %% Plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig,axs = plt.subplots(3,4,figsize=(6*phi,6))

for ax in axs.flatten():

    rand_img = np.random.randint(0,high=data_bin.shape[0])
    img = np.reshape(data_bin[rand_img,:],(28,28))
    ax.imshow(img,cmap='gray')
    ax.set_title('The number %i' %labels[rand_img])

plt.suptitle(f'How humans see the data\nSplit at {split_val}',fontsize=18)
plt.tight_layout(rect=[0,0,1,.95])

plt.savefig('figure14_code_challenge_14.png')

plt.show()

files.download('figure14_code_challenge_14.png')


In [ ]:
# %% Plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig,axs = plt.subplots(3,4,figsize=(6*phi,6))

for ax in axs.flatten():

    rand_img = np.random.randint(0,high=data_bin.shape[0])
    ax.plot(data_bin[rand_img,:],'ko')
    ax.set_title('The number %i' %labels[rand_img])

plt.suptitle(f'How the FFN model see the data\nSplit at {split_val}',fontsize=18)
plt.tight_layout(rect=[0,0,1,.95])

plt.savefig('figure15_code_challenge_14.png')

plt.show()

files.download('figure15_code_challenge_14.png')


In [ ]:
# %% Create train and test datasets

# Convert to tensor (float and integers)
data_tensor   = torch.tensor(data_bin).float()
labels_tensor = torch.tensor(labels).long()

# Split data with scikitlearn (10% test data)
train_data,test_data,train_labels,test_labels = train_test_split(data_tensor,labels_tensor,test_size=0.1)

# Convert to PyTorch datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Convert into DataLoader objects
batch_size   = 32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])


In [ ]:
# %% Function to generate the model

def gen_model():

    class mnist_FFN(nn.Module):
        def __init__(self):
            super().__init__()

            # Architecture
            self.input  = nn.Linear(784,64)
            self.fc1    = nn.Linear( 64,32)
            self.fc2    = nn.Linear( 32,32)
            self.output = nn.Linear( 32,10)

        # Forward propagation (log-softmax because NLLLoss instead of CrossEntropyLoss)
        def forward(self,x):

            x = F.relu(self.input(x))
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = torch.log_softmax( self.output(x),axis=1 )

            return x

    # Create model instance
    ANN = mnist_FFN()

    # Loss function
    loss_fun = nn.NLLLoss()

    # Optimizer (SGD to slow down learning for illustration purpose)
    optimizer = torch.optim.SGD(ANN.parameters(),lr=0.01)

    return ANN,loss_fun,optimizer


In [ ]:
# Test the model on one batch

ANN,loss_fun,optimizer = gen_model()

X,y  = next(iter(train_loader))
yHat = ANN(X)

# Print log-softmax output (size should be batch_size by output nodes)
print(yHat)
print(yHat.shape)
print()

# Print probabilities
print(torch.exp(yHat))
print()

# Compute loss
loss = loss_fun(yHat,y)
print('Loss: ')
print(loss)


In [ ]:
# %% Function to train the model

def train_model():

    # Parameters, model instance, inizialise vars
    num_epochs = 60
    ANN,loss_fun,optimizer = gen_model()

    losses    = []
    train_acc = []
    test_acc  = []

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Loop over training batches
        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            # Forward propagation and loss
            yHat = ANN(X)
            loss = loss_fun(yHat,y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Loss and accuracy from this batch
            batch_loss.append(loss.item())

            matches     = torch.argmax(yHat,axis=1) == y
            matches_num = matches.float()
            accuracy    = 100 * torch.mean(matches_num)
            batch_acc.append(accuracy)

        losses.append( np.mean(batch_loss) )
        train_acc.append( np.mean(batch_acc) )

        # Test accuracy
        ANN.eval()

        with torch.no_grad():
            X,y = next(iter(test_loader))
            yHat = ANN(X)
            test_acc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )

        ANN.train()

    return train_acc,test_acc,losses,ANN


In [ ]:
# %% Run the training

train_acc,test_acc,losses,ANN = train_model()


In [ ]:
# %% Plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig,ax = plt.subplots(1,2,figsize=(1.5*6*phi,6))

ax[0].plot(losses)
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].set_ylim([0,3])
ax[0].set_title('Model loss')

ax[1].plot(train_acc,label='Train accuracy')
ax[1].plot(test_acc,label='Test accuracy')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy (%)')
ax[1].set_ylim([10,100])
ax[1].set_title(f'Final model test accuracy: {test_acc[-1]:.2f}%\nSplit at {split_val}')
ax[1].legend()

plt.savefig('figure16_code_challenge_14.png')

plt.show()

files.download('figure16_code_challenge_14.png')


In [ ]:
# %% Exercise 1
#    Invert the images so that the numbers are black against a white background. It seems intuitive that the
#    model should perform just as well, but it's good to confirm. This is also a useful exercise to learn how
#    to invert images, which will become useful when learning about augmenting image datasets.

# The model does indeed perform similarly

# Flip data and run model as before (flip either binarised or normalised)
data_inverted_bin  = 1 - data_bin
data_inverted_norm = 1 - data_norm

# Plot
phi = ( 1 + np.sqrt(5) ) / 2
fig,axs = plt.subplots(3,4,figsize=(6*phi,6))

for ax in axs.flatten():

    rand_img = np.random.randint(0,high=data_inverted_bin.shape[0])
    img = np.reshape(data_inverted_bin[rand_img,:],(28,28))
    ax.imshow(img,cmap='gray')
    ax.set_title('The number %i' %labels[rand_img])

plt.suptitle(f'How humans see the data\nSplit at {split_val}',fontsize=18)
plt.tight_layout(rect=[0,0,1,.95])

plt.savefig('figure20_code_challenge_14_extra1.png')

plt.show()

files.download('figure20_code_challenge_14_extra1.png')


phi = ( 1 + np.sqrt(5) ) / 2
fig,axs = plt.subplots(3,4,figsize=(6*phi,6))

for ax in axs.flatten():

    rand_img = np.random.randint(0,high=data_inverted_norm.shape[0])
    img = np.reshape(data_inverted_norm[rand_img,:],(28,28))
    ax.imshow(img,cmap='gray')
    ax.set_title('The number %i' %labels[rand_img])

plt.suptitle('How humans see the data',fontsize=18)
plt.tight_layout(rect=[0,0,1,.95])

plt.savefig('figure21_code_challenge_14_extra1.png')

plt.show()

files.download('figure21_code_challenge_14_extra1.png')
